In [1]:
import numpy as np
import pandas as pd

# Load dataset
def load_dataset(file_path):
    dataset = pd.read_csv(file_path)
    return dataset

# Train-test split
def train_test_split(dataset, test_size=0.2):
    num_samples = len(dataset)
    test_samples = int(test_size * num_samples)
    
    test_data = dataset.iloc[:test_samples]
    train_data = dataset.iloc[test_samples:]
    
    return train_data, test_data

# Example usage:
dataset = load_dataset('dataset.csv')
train_data, test_data = train_test_split(dataset)


In [2]:
# Extract labels and features
def extract_labels_features(data):
    labels = data['label'].values
    features = data.drop('label', axis=1).values
    return labels, features

# Example usage:
train_labels, train_features = extract_labels_features(train_data)
test_labels, test_features = extract_labels_features(test_data)

# Normalize pixel values
train_features = train_features / 255.0
test_features = test_features / 255.0

# Reshape features for CNN input
train_features = train_features.reshape(-1, 128, 128, 1)
test_features = test_features.reshape(-1, 128, 128, 1)


In [3]:
# Define the CNN architecture
class CNN:
    def __init__(self):
        self.layers = [
            ConvolutionLayer(num_filters=16, filter_size=3),
            ActivationLayer(),
            PoolingLayer(pool_size=2),
            FlattenLayer(),
            FullyConnectedLayer(input_size=16*63*63, output_size=128),
            ActivationLayer(),
            FullyConnectedLayer(input_size=128, output_size=64),
            ActivationLayer(),
            FullyConnectedLayer(input_size=64, output_size=10)  # Assuming 10 classes
        ]

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

# Define layer classes
class ConvolutionLayer:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / (filter_size * filter_size)

    def forward(self, inputs):
        self.inputs = inputs
        self.input_shape = inputs.shape

        batch_size, height, width, num_channels = inputs.shape
        output_height = height - self.filter_size + 1
        output_width = width - self.filter_size + 1

        outputs = np.zeros((batch_size, output_height, output_width, self.num_filters))

        for y in range(output_height):
            for x in range(output_width):
                for filter_num in range(self.num_filters):
                    outputs[:, y, x, filter_num] = np.sum(
                        inputs[:, y:y+self.filter_size, x:x+self.filter_size, :] * self.filters[filter_num, :],
                        axis=(1, 2, 3)
                    )

        return outputs

# Define Activation layer
class ActivationLayer:
    def forward(self, inputs):
        self.inputs = inputs
        return np.maximum(0, inputs)

# Define Pooling layer
class PoolingLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, inputs):
        self.inputs = inputs
        batch_size, height, width, num_channels = inputs.shape
        output_height = height // self.pool_size
        output_width = width // self.pool_size

        outputs = np.zeros((batch_size, output_height, output_width, num_channels))

        for y in range(output_height):
            for x in range(output_width):
                for c in range(num_channels):
                    outputs[:, y, x, c] = np.max(
                        inputs[:, y*self.pool_size:(y+1)*self.pool_size, x*self.pool_size:(x+1)*self.pool_size, c])

        return outputs

# Define Flatten layer
class FlattenLayer:
    def forward(self, inputs):
        self.inputs_shape = inputs.shape
        return inputs.reshape(len(inputs), -1)

# Define Fully connected layer
class FullyConnectedLayer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros(output_size)

    def forward(self, inputs):
        self.inputs = inputs
        return np.dot(inputs, self.weights) + self.biases

# Instantiate the CNN
cnn_model = CNN()


In [ ]:
# Define training loop
def train_model(model, train_features, train_labels, epochs, learning_rate):
    for epoch in range(epochs):
        # Forward pass
        predictions = model.forward(train_features)

        # Backpropagation
        loss = calculate_loss(predictions, train_labels)
        gradients = calculate_gradients(predictions, train_labels)
        update_parameters(model, gradients, learning_rate)

        # Print loss
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

# Define loss function
def calculate_loss(predictions, targets):
    loss = np.mean(np.square(predictions - targets))
    return loss

# Define gradient calculation
def calculate_gradients(predictions, targets):
    gradients = 2 * (predictions - targets) / len(targets)
    return gradients

# Define parameter update
def update_parameters(model, gradients, learning_rate):
    for layer in model.layers:
        if hasattr(layer, 'weights'):
            layer.weights -= learning_rate * gradients

# Train the model
train_model(cnn_model, train_features, train_labels, epochs=100, learning_rate=0.01)
